# GPU计算
到⽬前为⽌，我们⼀直在使⽤CPU计算。对复杂的神经⽹络和⼤规模的数据来说，使⽤CPU来计
算可能不够⾼效。在本节中，我们将介绍如何使⽤单块NVIDIA GPU来计算。⾸先，需要确保已
经安装好了⾄少⼀块NVIDIA GPU。然后，下载CUDA并按照提⽰设置好相应的路径（可参考附录
中“使⽤AWS运⾏代码” ⼀节）。这些准备⼯作都完成后，下⾯就可以通过nvidia-smi命令来查
看显卡信息了。

In [1]:
!nvidia-smi # 对Linux/macOS⽤⼾有效

'nvidia-smi' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


接下来，我们需要确认安装了MXNet的GPU版本。安装⽅法⻅“获取和运⾏本书的代码” ⼀节。
运⾏本节中的程序需要⾄少2块GPU。
## 计算设备
MXNet可以指定⽤来存储和计算的设备，如使⽤内存的CPU或者使⽤显存的GPU。默认情况下，
MXNet会将数据创建在内存，然后利⽤CPU来计算。在MXNet中， mx.cpu()（或者在括号⾥填任
意整数）表⽰所有的物理CPU和内存。这意味着， MXNet的计算会尽量使⽤所有的CPU核。但mx.
gpu()只代表⼀块GPU和相应的显存。如果有多块GPU，我们⽤mx.gpu(i)来表⽰第i块GPU及
相应的显存（i从0开始）且mx.gpu(0)和mx.gpu()

In [4]:
import mxnet as mx
from mxnet import nd
from mxnet.gluon import nn
mx.cpu(), mx.gpu(), mx.gpu(1)

(cpu(0), gpu(0), gpu(1))

## NDArray的GPU计算
在默认情况下， NDArray存在内存上。因此，之前我们每次打印NDArray的时候都会看
到@cpu(0)这个标识。

In [5]:
x = nd.array([1, 2, 3])
x


[1. 2. 3.]
<NDArray 3 @cpu(0)>

我们可以通过NDArray的context属性来查看该NDArray所在的设备。

In [6]:
x.context

cpu(0)

### GPU上的存储
我们有多种⽅法将NDArray存储在显存上。例如，我们可以在创建NDArray的时候通过ctx参数
指定存储设备。下⾯我们将NDArray变量a创建在gpu(0)上。注意，在打印a时，设备信息变成
了@gpu(0)。创建在显存上的NDArray只消耗同⼀块显卡的显存。我们可以通过nvidia-smi命
令查看显存的使⽤情况。通常，我们需要确保不创建超过显存上限的数据

In [9]:
a = nd.array([1, 2, 3], ctx=mx.gpu())
a


[1. 2. 3.]
<NDArray 3 @cpu(0)>

假设⾄少有2块GPU，下⾯代码将会在gpu(1)上创建随机数组

In [ ]:
B = nd.random.uniform(shape=(2, 3), ctx=mx.gpu(1))
B

除了在创建时指定，我们也可以通过copyto函数和as_in_context函数在设备之间传输数据。
下⾯我们将内存上的NDArray变量x复制到gpu(0)上。

In [11]:
y = x.copyto(mx.gpu())
y


[1. 2. 3.]
<NDArray 3 @cpu(0)>

In [ ]:
z = x.as_in_context(mx.gpu())
z

需要区分的是，如果源变量和⽬标变量的context⼀致， as_in_context函数使⽬标变量和源
变量共享源变量的内存或显存。

In [ ]:
y.as_in_context(mx.gpu()) is y

而copyto函数总是为⽬标变量开新的内存或显存

In [ ]:
y.copyto(mx.gpu()) is y

### GPU上的计算
MXNet的计算会在数据的context属性所指定的设备上执⾏。为了使⽤GPU计算，我们只需要事
先将数据存储在显存上。计算结果会⾃动保存在同⼀块显卡的显存上。

In [ ]:
(z+2).exp()*y

注意，MXNet要求计算的所有输⼊数据都在内存或同⼀块显卡的显存上。这样设计的原因是CPU和
不同的GPU之间的数据交互通常⽐较耗时。因此， MXNet希望⽤⼾确切地指明计算的输⼊数据都
在内存或同⼀块显卡的显存上。例如，如果将内存上的NDArray变量x和显存上的NDArray变
量y做运算，会出现错误信息。当我们打印NDArray或将NDArray转换成NumPy格式时，如果数
据不在内存⾥， MXNet会将它先复制到内存，从而造成额外的传输开销。
## Gluon的GPU计算
同NDArray类似， Gluon的模型可以在初始化时通过ctx参数指定设备。下⾯的代码将模型参数
初始化在显存上。

In [ ]:
net = nn.Sequential()
net.add(nn.Dense(1))
net.initialize(ctx=mx.gpu())

当输⼊是显存上的NDArray时， Gluon会在同⼀块显卡的显存上计算结果。

In [ ]:
net(y)

下⾯我们确认⼀下模型参数存储在同⼀块显卡的显存上

In [ ]:
net[0].weight.data()

## ⼩结
- MXNet可以指定⽤来存储和计算的设备，如使⽤内存的CPU或者使⽤显存的GPU。在默认
情况下, MXNet会将数据创建在内存，然后利⽤CPU来计算。
- MXNet要求计算的所有输⼊数据都在内存或同⼀块显卡的显存上。